In [1]:
import json
import pandas as pd

data = pd.read_json('./train.json')
print(len(data))
print(data.head(3))

9397
   岗位ID                                               解析结果  简历ID
0   519  {'profileEduExps': [{'schoolLevel': [], 'schoo...     0
1   418  {'profileEduExps': [{'schoolLevel': ['14844587...     1
2   516  {'profileEduExps': [{'schoolLevel': ['62 63 50...     2


In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

data = pd.read_json('./train.json')
data['解析结果'] = data['解析结果'].apply(lambda x : json.dumps(x).replace('"', ' ').replace('"', ' ').split())

train_data, test_data = train_test_split(data, train_size=0.75)

joblist = pd.read_json('./job_list.json')
joblist['解析结果'] = joblist['岗位名称'] + ' ' + joblist['岗位介绍'] + ' ' + joblist['岗位要求']
joblist['解析结果'] = joblist['解析结果'].apply(lambda x : x.split())



In [4]:
train_feat = []
for row in train_data.iterrows():
    label = row[1]['岗位ID']
    query_text= row[1]['解析结果']
    feat = [
        label,
        len(query_text), len(set(query_text)), len(query_text) - len(set(query_text)),
    ]
    for target_text in joblist['解析结果']:
        feat += [
            len(set(query_text) & set(target_text)),
            len(set(query_text) & set(target_text)) / len(query_text),
            len(set(query_text) & set(target_text)) / len(target_text),
            
            len(set(query_text) & set(target_text)) / len(set(target_text)),
            len(set(query_text) & set(target_text)) / len(set(query_text))

        ]
    train_feat.append(feat)
train_feat = np.array(train_feat)
m = RandomForestClassifier()
m.fit(
    train_feat[:, 1:],
    train_feat[:, 0],
)

test_feat = []
labels = []
for row in test_data.iterrows():
    labels.append(row[1]['岗位ID'])
    query_text= row[1]['解析结果']
    feat = [
        len(query_text), len(set(query_text)), len(query_text) - len(set(query_text)),
    ]
    for target_text in joblist['解析结果']:
        feat += [
            len(set(query_text) & set(target_text)),
            len(set(query_text) & set(target_text)) / len(query_text),
            len(set(query_text) & set(target_text)) / len(target_text),
            
            len(set(query_text) & set(target_text)) / len(set(target_text)),
            len(set(query_text) & set(target_text)) / len(set(query_text))

        ]
    test_feat.append(feat)
test_feat = np.array(test_feat)
# pd.DataFrame({
#     '简历ID': range(len(test_data)),
#     '岗位ID': m.predict(test_feat).astype(int)
# }).to_csv('./result.csv', index=None)




pred = m.predict(test_feat).astype(int)
print(labels)
print(pred)
F1 = f1_score(labels,pred,average='macro')
print(F1)




[519, 412, 418, 418, 516, 516, 516, 332, 331, 328, 340, 328, 427, 516, 418, 418, 328, 430, 518, 418, 342, 418, 418, 328, 519, 519, 516, 328, 418, 418, 342, 519, 418, 418, 406, 516, 518, 518, 516, 517, 516, 328, 418, 328, 328, 418, 326, 418, 519, 418, 331, 518, 342, 418, 518, 332, 519, 518, 342, 328, 519, 418, 518, 517, 418, 331, 328, 417, 418, 328, 412, 430, 340, 328, 331, 519, 518, 517, 516, 418, 328, 418, 331, 412, 519, 517, 418, 418, 517, 519, 517, 331, 418, 328, 328, 418, 412, 417, 418, 328, 328, 418, 418, 328, 519, 331, 516, 335, 518, 340, 418, 331, 326, 519, 328, 342, 519, 412, 418, 516, 430, 412, 517, 331, 418, 418, 412, 328, 516, 406, 417, 418, 328, 418, 418, 418, 412, 412, 519, 519, 331, 328, 331, 418, 430, 412, 328, 417, 331, 418, 340, 518, 418, 412, 418, 412, 412, 418, 340, 417, 412, 517, 519, 417, 418, 328, 342, 328, 516, 518, 328, 328, 340, 328, 328, 519, 418, 519, 328, 328, 519, 328, 324, 340, 418, 517, 418, 328, 418, 516, 516, 519, 517, 340, 328, 418, 331, 519, 418, 430,